### BreakOut Room Study Case

Dalam Study case kali ini, data yang digunakan adalah data transportation_sales.db yang merupakan data tentang penjualan beberapa jenis kendaraan di beberapa cabang toko di berbagai negara.

####Gdrive Connection

In [16]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [17]:
#  Membuat connection ke database yang udah ada
import os
import pandas as pd
import sqlite3 as sq
path = "gdrive/MyDrive/MBKM/Week 4 - SQL/"
os.listdir(path)

conn = sq.connect(path + 'transportation_sales.db')
cur = conn.cursor()

## Cek Tabel terlebih dahulu

In [18]:
cur.execute("""
              SELECT 
                  name 
              FROM sqlite_master 
              WHERE type='table'
            """)

print(cur.fetchall())

[('Customers',), ('Products',), ('Orders',), ('mytable',)]


#### Melihat isi tabel masing-masing

In [19]:
pd.read_sql_query("Select * from customers limit 3",conn)

,Customer_ID,CUSTOMERNAME,PHONE,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY
0,1,Land of Toys Inc.,2125557818,897 Long Airport Avenue,None,NYC,NY,10022,USA
1,2,Reims Collectables,26.47.1555,59 rue de l'Abbaye,None,Reims,None,51100,France
2,3,Lyon Souveniers,+33 1 46 62 7555,27 rue du Colonel Pierre Avia,None,Paris,None,75508,France


In [20]:
pd.read_sql_query("Select * from orders limit 10",conn)

,index,ORDER_ID,CUSTOMER_ID,PRODUCT_ID,QUANTITYORDERED,SALES,ORDERDATE,STATUS
0,0,10107,3,2,30,2871.00,2003-01-29,On Hold
1,1,10121,1,4,34,2765.90,2003-02-24,Shipped
2,2,10134,4,9,41,3884.34,2003-03-24,Cancelled
3,3,10145,2,10,45,3746.70,2003-05-07,Shipped
4,4,10159,7,5,49,5205.27,2003-05-28,Shipped
5,5,10168,8,11,36,3479.76,2003-07-01,Cancelled
6,6,10180,2,10,29,2497.77,2003-07-24,On Hold
7,7,10188,9,8,48,5512.32,2003-08-25,Shipped
8,8,10201,8,20,22,2168.54,2003-10-10,Shipped
9,9,10211,5,14,41,4708.44,2003-10-28,Shipped


In [21]:
pd.read_sql_query("Select * from products limit 10",conn)

,PRODUCT_ID,PRODUCTCODE,PRODUCTLINE,MSRP
0,1,S10_1678,Motorcycles,95
1,2,S10_1949,Classic Cars,214
2,3,S12_1666,Trucks and Buses,136
3,4,S12_2823,Motorcycles,150
4,5,S12_3148,Classic Cars,151
5,6,S18_1129,Classic Cars,141
6,7,S18_1342,Vintage Cars,102
7,8,S18_2238,Classic Cars,163
8,9,S18_2248,Vintage Cars,60
9,10,S18_2319,Trucks and Buses,122


#### Case 1

Pemilik toko mengalami kesulitan saat melihat tabel Orders karena informasi customer dan product hanya berupa ID, sedangkan pemilik toko ingin melihat nama customer beserta kode dan tipe kendaraan yang dibeli

In [22]:
customer = pd.read_sql_query("""
            SELECT C.CUSTOMER_ID, C.CUSTOMERNAME, P.PRODUCTCODE, P.PRODUCTLINE
            FROM CUSTOMERS C, PRODUCTS P, ORDERS O
            WHERE C.CUSTOMER_ID = O.CUSTOMER_ID
            AND P.PRODUCT_ID = O.PRODUCT_ID
            """,conn)
customer

,Customer_ID,CUSTOMERNAME,PRODUCTCODE,PRODUCTLINE
0,3,Lyon Souveniers,S10_1949,Classic Cars
1,1,Land of Toys Inc.,S12_2823,Motorcycles
2,4,Toys4GrownUps.com,S18_2248,Vintage Cars
3,2,Reims Collectables,S18_2319,Trucks and Buses
4,7,Daedalus Designs Imports,S12_3148,Classic Cars
5,8,Herkku Gifts,S18_2325,Vintage Cars
6,2,Reims Collectables,S18_2319,Trucks and Buses
7,9,Mini Wheels Co.,S18_2238,Classic Cars
8,5,Corporate Gift Ideas Co.,S18_2949,Vintage Cars
9,7,Daedalus Designs Imports,S18_2432,Trucks and Buses


#### Case 2

Pemilik toko ingin mengetahui apakah terjadi peningkatan atau bahkan penurunan baik jumlah customer maupun jumlah transaksi pembelian di setiap tahunnya, selain itu ingin mengetahui apakah terjadi New Purchase (Pembelian Baru) dari New Customer. Untuk mengetahui hal tersebut, pertama-tama kita harus mendapatkan berapa jumlah uniq customer di setiap tahunnya.

In [23]:
#melihat unique customer
customercheck = pd.read_sql_query("""
SELECT o.ORDERDATE,
       COUNT(DISTINCT o.CUSTOMER_ID) AS unique_customers
FROM orders o
JOIN customers c ON o.CUSTOMER_ID = c.Customer_ID
GROUP BY o.ORDERDATE
ORDER BY o.ORDERDATE
""",conn)
customercheck

,ORDERDATE,unique_customers
0,2003-01-29,1
1,2003-02-24,1
2,2003-03-24,1
3,2003-05-07,1
4,2003-05-28,1
5,2003-07-01,1
6,2003-07-24,1
7,2003-08-25,1
8,2003-10-10,1
9,2003-10-28,1


#### Case 3

Sebutkan top 3 product pada tahun 2003. Apakah dari produk-produk tersebut ada yang menjadi favorit dalam 3 tahun berturut-turut?

In [24]:
top3 = pd.read_sql_query("""
                        SELECT O.PRODUCT_ID, P.PRODUCTCODE, P.PRODUCTLINE, SUM(O.QUANTITYORDERED) AS TOTAL_QUANTITY
                        FROM ORDERS O, PRODUCTS P
                        WHERE O.ORDERDATE BETWEEN '2003-01-01' AND '2003-12-31' 
                        GROUP BY O.PRODUCT_ID
                        ORDER BY TOTAL_QUANTITY DESC
                        LIMIT 3
""",conn)
top3

,PRODUCT_ID,PRODUCTCODE,PRODUCTLINE,TOTAL_QUANTITY
0,10,S10_1678,Motorcycles,1480
1,5,S10_1678,Motorcycles,980
2,8,S10_1678,Motorcycles,960


#### Case 4

Dapatkan data total penjualan dengan status order dalam proses pengiriman (shipped), ditahan (on hold), dan dibatalkan (cancelled) untuk setiap customer.

In [25]:
total_sales = pd.read_sql_query("""
                                SELECT SUM(SALES) as total_penjualan
                                FROM ORDERS
                                WHERE STATUS = 'Cancelled'
                                OR STATUS = 'Shipped'
                                OR STATUS = 'On Hold'               
""",conn)
total_sales

,total_penjualan
0,116907.81


#### Case 5

Berapa jumlah order quantity yang terjual masing-masing produk pada tahun 2004?

In [26]:
orderquantity = pd.read_sql_query("""
                                SELECT PRODUCT_ID, SUM(QUANTITYORDERED) AS Total_order
                                FROM ORDERS
                                WHERE ORDERDATE BETWEEN '2004-01-01' AND '2004-31-12'  
                                GROUP BY (PRODUCT_ID)
                                ORDER BY PRODUCT_ID
""",conn)

orderquantity

,PRODUCT_ID,Total_order
0,1,20
1,6,21
2,7,36
3,12,24
4,13,41
5,14,146
6,15,42
7,16,42
8,17,23
9,18,86


Pemilik toko ingin mengetahui hasil penjualan di setiap tahunnya. Oleh karena itu dari tabel Orders, berapakah pendapatan di setiap tahun?

In [27]:
year_sale = pd.read_sql_query("""
                    SELECT strftime('%Y', ORDERDATE) AS YEAR, SUM(SALES) AS TOTAL_SALES
FROM ORDERS
GROUP BY year

""",conn)
year_sale

,YEAR,TOTAL_SALES
0,2003,46327.46
1,2004,50779.54
2,2005,27316.89


#### Case 6

Pada tahun 2003 Pemilik toko menjumpai anomali pada database, Pemilik toko mendapati masih terdapat order yang berstatus "On Hold", untuk menyelidiki hal tersebut pemilik toko ingin seluruh informasi order pada tahun 2003 yang berstatus "On Hold".

In [28]:
holditem = pd.read_sql_query("""
                            SELECT * FROM ORDERS
                            WHERE STATUS = 'On Hold' 
                            AND ORDERDATE BETWEEN '2003-01-01' AND '2003-12-31'
""",conn)
holditem

,index,ORDER_ID,CUSTOMER_ID,PRODUCT_ID,QUANTITYORDERED,SALES,ORDERDATE,STATUS
0,0,10107,3,2,30,2871.00,2003-01-29,On Hold
1,6,10180,2,10,29,2497.77,2003-07-24,On Hold
